In [ ]:
import asyncio
import httpx
import random
import string
import logging
import argparse
from datetime import datetime
from collections import Counter

# --- Configuration & Setup ---

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

BASE_API_URL = "https://api.cloudflareclient.com/v0a{}/reg"

# --- Helper Functions ---

def gen_random_string(length: int) -> str:
    """Generates a random alphanumeric string of a given length."""
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))

def get_request_headers() -> dict:
    """Returns headers required by the Cloudflare API."""
    return {
        "Content-Type": "application/json; charset=UTF-8",
        "Host": "api.cloudflareclient.com",
        "Connection": "Keep-Alive",
        "Accept-Encoding": "gzip",
        "User-Agent": "okhttp/3.12.1",
    }

def build_request_body(referrer_id: str) -> dict:
    """Constructs the JSON body for the API request."""
    install_id = gen_random_string(22)
    return {
        "key": f"{gen_random_string(43)}=",
        "install_id": install_id,
        "fcm_token": f"{install_id}:APA91b{gen_random_string(134)}",
        "referrer": referrer_id,
        "warp_enabled": False,
        "tos": datetime.utcnow().replace(microsecond=0).isoformat() + "Z",
        "type": "Android",
        "locale": "es_ES",
    }

# --- Core Logic ---

async def run_worker(session: httpx.AsyncClient, referrer_id: str, results_counter: Counter):
    """Handles a single API request and logs the result."""
    try:
        api_version = ''.join(random.choices(string.digits, k=3))
        url = BASE_API_URL.format(api_version)

        headers = get_request_headers()
        body = build_request_body(referrer_id)

        response = await session.post(url, json=body, headers=headers, timeout=10)

        if response.status_code == 200:
            logging.info("SUCCESS: Referrer profile updated. +1GB of data added.")
            results_counter.update(["success"])
        else:
            logging.error(f"FAILURE: Status code {response.status_code}, Response: {response.content}")
            results_counter.update(["failure"])

    except httpx.TimeoutException:
        logging.error("Request timed out.")
        results_counter.update(["failure"])
    except httpx.RequestError as e:
        logging.error(f"HTTP Request Error: {e}")
        results_counter.update(["failure"])
    except Exception as e:
        logging.error(f"Unexpected error: {e}")
        results_counter.update(["failure"])

# --- Main Execution ---

async def main(referrer_id: str, total_requests: int, delay_seconds: float):
    """Orchestrates all requests using an HTTP session and counters."""
    results = Counter()

    async with httpx.AsyncClient() as session:
        tasks = []
        for i in range(total_requests):
            logging.info(f"Scheduling request {i + 1} of {total_requests}.")
            task = asyncio.create_task(run_worker(session, referrer_id, results))
            tasks.append(task)
            await asyncio.sleep(delay_seconds)

        await asyncio.gather(*tasks)

    logging.info("--- All requests completed ---")
    logging.info(f"Total successes: {results['success']} GB")
    logging.info(f"Total failures: {results['failure']}")

# --- CLI Entry Point ---

if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        description="Cloudflare WARP+ Referral Booster",
        formatter_class=argparse.RawTextHelpFormatter,
        epilog="""Example:
  python warp_referral_booster.py YOUR_CLIENT_ID -n 10 -d 5
  Sends 10 requests with 5 seconds delay between each."""
    )

    parser.add_argument(
        "referrer_id",
        help="Your WARP+ Client ID (referrer)."
    )
    parser.add_argument(
        "-n", "--requests",
        type=int,
        default=10,
        help="Total number of requests to send. (default: 10)"
    )
    parser.add_argument(
        "-d", "--delay",
        type=float,
        default=20.0,
        help="Delay between requests in seconds. (default: 20.0)"
    )

    args = parser.parse_args()

    try:
        asyncio.run(main(args.referrer_id, args.requests, args.delay))
    except KeyboardInterrupt:
        logging.info("Process interrupted by user. Exiting.")


usage: colab_kernel_launcher.py [-h] [-n REQUESTS] [-d DELAY] referrer_id
colab_kernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2